In [2]:
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras.activations import relu, sigmoid
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MultiLabelBinarizer
from keras.optimizers import Adam
import tensorflow as tf
import os
import math
import numpy as np
import cv2 as cv
import random as rd
import matplotlib.pyplot as plt


rd.seed(1)

In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', input_shape=(96, 96, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(3, 3),  # Menggunakan filter pooling yang lebih besar
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(17, activation='sigmoid')
])


In [4]:
class FashionDataset(Sequence):
    def __init__(self, paths: list, batch_size: int):
        super(FashionDataset, self).__init__()
        self.batch_size = batch_size
        self.data_paths = paths
        self.classes = []
        self.labels = self._generate_labels(self.data_paths)

    def __len__(self):
        return math.ceil(len(self.data_paths) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.data_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        return np.array([self._load_image(i) for i in batch_x]), np.array(batch_y)

    def _generate_labels(self, paths: list):
        labels = []
        for path in paths:
            label = path.split(os.path.sep)[-2].split('_')
            labels.append(label)

        mlb = MultiLabelBinarizer()
        labels = mlb.fit_transform(labels)
        self.classes = mlb.classes_

        return labels

    @staticmethod
    def _load_image(path: str):
        img = cv.imread(path)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        img = cv.resize(img, (96, 96))
        return img / 255.0


In [5]:
train_sz = 0.75
paths = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename))

paths = sorted(paths)
rd.shuffle(paths)
assert 0.0 < train_sz <= 1.0
thresh = round(len(paths) * train_sz)
train_paths = paths[:thresh]
test_paths = paths[thresh:]
print(train_paths[0], test_paths[1])


/kaggle/input/apparel-dataset/blue_shirt/102.jpg /kaggle/input/apparel-dataset/black_shorts/31.jpg


In [6]:
train_x = FashionDataset(train_paths, batch_size=32)
test_y = FashionDataset(test_paths, batch_size=32)

model.compile(
    loss=tf.keras.losses.MAE,
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-3, decay=0.00005),
    metrics=['accuracy']
)

In [7]:
for i in range(len(train_x.classes)):
    classes = '\n'.join(train_x.classes)

with open('classes.txt', 'w') as f:
    f.write(classes)
    
model_json = model.to_json()

with open('model.json', 'w') as f:
    f.write(model_json)

In [9]:
best_w = ModelCheckpoint('best.h5',
                        monitor='val_loss',
                        verbose=1,
                        save_weights_only=True,
                        save_best_only=True,
                        mode='min',
                        save_freq='epoch')

last_w = ModelCheckpoint('last.h5',
                        monitor='val_loss',
                        verbose=1,
                        save_weights_only=True,
                        save_best_only=False,
                        mode='auto',
                        save_freq='epoch')

In [ ]:
with tf.device('/GPU:0'):
    history = model.fit(x=train_x,
                        validation_data=test_y,
                        steps_per_epoch=len(train_x),
                        epochs=100,
                        callbacks=[best_w, last_w])

Epoch 1/100
379/379 [==============================] - ETA: 0s - loss: 0.1246 - accuracy: 0.3994
Epoch 1: val_loss improved from inf to 0.09254, saving model to best.h5

Epoch 1: saving model to last.h5
379/379 [==============================] - 450s 1s/step - loss: 0.1246 - accuracy: 0.3994 - val_loss: 0.0925 - val_accuracy: 0.2147
Epoch 2/100
379/379 [==============================] - ETA: 0s - loss: 0.0491 - accuracy: 0.4806
Epoch 2: val_loss improved from 0.09254 to 0.05213, saving model to best.h5

Epoch 2: saving model to last.h5
379/379 [==============================] - 434s 1s/step - loss: 0.0491 - accuracy: 0.4806 - val_loss: 0.0521 - val_accuracy: 0.5136
Epoch 3/100
379/379 [==============================] - ETA: 0s - loss: 0.0422 - accuracy: 0.5087
Epoch 3: val_loss did not improve from 0.05213

Epoch 3: saving model to last.h5
379/379 [==============================] - 430s 1s/step - loss: 0.0422 - accuracy: 0.5087 - val_loss: 0.0549 - val_accuracy: 0.4822
Epoch 4/100
379/